In [169]:
import numpy
import matplotlib
import pandas
import os
import math
import hashlib
from itertools import permutations

# Task 1
## Rotate Left Function 

### Explanation:
- The bits go past the left and around to the right.
- This creates a circle shift.
- 4-bit (`0b1111`) to keep results within 4 bits.

## Rotate Right Function
- This is just a reverse of rotate left.
- Quite useful in cryptographic operations that require predictable bit-level editing.

## Choice Function
- Used to choose bits from 'y' or 'z' based on the value of 'x'.
- If a bit in 'x' is 1, take the bit from 'y', if it's 0, take it from 'z'.
- Used in SHA-256 and other hash functions.

## Majority Function
- Looks at each bit and picks the value that appears in at least two of the three inputs.
- Similar to a "voting system" for each bit - if two or more inputs agree, thats the winning value.
- It is used in hash functions to mix bits and make the output harder to predict.

In [170]:
def rotl(x, n=1):
    # Rotate bits of a 32-bit unsigned integer to the left by n positions
    n = n % 4  
    return ((x << n) & 0b1111) | (x >> (4 - n))  

def rotr(x, n=1):
    # Rotate bits of a 4-bit unsigned integer to the right by n positions
    n = n % 4  
    return ((x >> n) | (x << (4 - n))) & 0b1111   

def ch(x, y, z):
    # Choose bits from y where x is 1, and from z where x is 0
    return (x & y) ^ (~x & z)

def maj(x, y, z):
     # Choose 1 if at least two of x, y, or z are 1
    return (x & y) | (x & z) | (y & z)

In [171]:
x, y, z = 0b1100, 0b1010, 0b0110
rotated_x = rotl(x, 1)  # Rotate left by 1 bit

print(f"Original: {bin(x)} ({x})")
print(f"x = {bin(x)} ({x})")
print(f"y = {bin(y)} ({y})")
print(f"z = {bin(z)} ({z})")

# Test rotate left
print(f"Rotated Left: {bin(rotated_x)} ({rotated_x})")

# Test rotate right
rotr_x = rotr(x, 1)
print(f"Rotated Right: {bin(rotr_x)} ({rotr_x})")

# Test ch function
ch_result = ch(x, y, z)
print(f"ch(x, y, z) = {bin(ch_result)} ({ch_result})")

# Test maj function
maj_result = maj(x, y, z)
print(f"maj(x, y, z) = {bin(maj_result)} ({maj_result})")

Original: 0b1100 (12)
x = 0b1100 (12)
y = 0b1010 (10)
z = 0b110 (6)
Rotated Left: 0b1001 (9)
Rotated Right: 0b110 (6)
ch(x, y, z) = 0b1010 (10)
maj(x, y, z) = 0b1110 (14)


## Reference:
- Lab materials/hash_functions.ipynb
# END TO TASK 1
## Task 2: Conversion, C - Python

### Explanation:
- This task requires me to convert a basic C-style hash function to Python.
- It loops over each character in a string and builds a running hash value.
- 'ord(char)' gets the Unicode value of each character.
- The result is taken modulo 101 to limit it to a fixed range.

### Test Case:
- I tested the function on a set of strings.
- The outputs match expectations and show how different inputs map to different hash values.

In [172]:
def hash_function(s):
    hashval = 0 # Start with a hash value of 0
    for char in s:
        # For each char, update the hash value using its Unicode and a multiplier
        hashval = ord(char) + 31 * hashval  
    # Keep the hask value in a fixed ranghe (0-100)
    return hashval % 101 

In [173]:
# Test cases with expected outputs
test_strings = ["hello", "world", "hash", "function", "test", "python"]

# Print each string with its corresponding hash value
for s in test_strings:
    print(f"hash_function('{s}') = {hash_function(s)}")

hash_function('hello') = 17
hash_function('world') = 34
hash_function('hash') = 15
hash_function('function') = 100
hash_function('test') = 86
hash_function('python') = 91


## Reference:
- Lab materials/hash_functions.ipynb
- Wikipedia. (n.d.). Hash function. Retrieved from https://en.wikipedia.org/wiki/Hash_function
# END TO TASK 2
## Task 3: SHA256

### Explanation:
- This task is to show how SHA-256 prepares messages before hashing.
- The input file is read as bytes
- Padding starts with a '1' bit, followed by enough '0' bits to make the message length 64 bits short of a 
  multiple of 512.
- Then the length of the original input is added as a 64-bit big-endian integer.
- This makes sure the message is the correct size for SHA-256's algorithm to process.

In [174]:
def sha256_pad(file_path):
    # Opens the file in binary mode and reads whats inside
    with open(file_path, 'rb') as f:
        original = f.read()

    # Gets the length of the original message in bits
    length = len(original) * 8  # length in bits
    data = original + b'\x80'

    # Add '0' bits until the total length is 64 bits short of a multiple of 512
    while (len(data) * 8) % 512 != 448:
        data += b'\x00'

    # Append the origial message length as a 64-bit int
    data += length.to_bytes(8, 'big')

    # Extract only the padding part for display
    padding = data[len(original):]

    # Prints padding bytes in hex format
    print(' '.join(f'{b:02x}' for b in padding))

In [175]:
# Create the test file with some content
with open("test.txt", "w") as f:
    f.write("abc")

# Then run the padding function
sha256_pad("test.txt")

80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


# Reference:
- Wikipedia. (n.d.). SHA-2. Retrieved from https://en.wikipedia.org/wiki/SHA-2
- Lab materials/sha256.ipynb
# END TO TASK 3
## Task 4: Prime Numbers

### Explanation: Calculate the first 100 prime numbers using two different algorithms:
### Method 1: Trial Division
This method will check if a number is divisible by any number from 2 - √n
If it is not divisible, it's considered prime. This method is slower but simpler

### Method 2: Sieve of Eratosthenes
This method creates a list of possible primes and removes multiples of each number.
More efficient and better for generating many primes.

In [176]:
def is_prime(n):
    # Numbers that are less than 2 are not prime
    if n < 2:
        return False
    
    # Checks divisibility from 2 up to √n
    for i in range(2, int(n**0.5) + 1):
        # If divisible by any i, it's also not a prime
        if n % i == 0:
            return False
        
        # If no divisors found, it's a prime :)
        return True

In [177]:
def primes_trial_division(limit):
    primes = []                     # List to hold prime numbers
    num = 2                         # Starting number to check for prime

    # Continues to find primes until we reach the desired count
    while len(primes) < limit:
        if is_prime(num):           # Use helepr to test for primality
            primes.append(num)      # If prime, add to list
        num += 1                    # Check the next number
    return primes

In [178]:
def sieve_of_eratosthenes(n):
    size = 600                                 # Upper bound for range to find primes
    is_prime = [True] * size                   # Assumes all nymbers are prime at the start
    is_prime[0] = is_prime[1] = False          # 0 and 1 are not prime

    # Loop through the range and mark non-primes
    for i in range(2, int(size**0.5) + 1):
        if is_prime[i]:
            # Mark all multiples of i as non=prime
            for j in range(i*i, size, i):
                is_prime[j] = False
    
    # Return the first n prime numbers
    primes = [i for i, val in enumerate(is_prime) if val]
    return primes[:n]

In [179]:
print("Trial Division:\n", primes_trial_division(100))
print("\nSieve of Eratosthenes:\n", sieve_of_eratosthenes(100))

Trial Division:
 [5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203]

Sieve of Eratosthenes:
 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


### Comparing the Results:
Both methods do the job, they correctly generate the first 100 prime numbers.
But Sieve of Eratosthenes is faster for bigger listrs, but Trial Division is much easier to understand.

### Reference:
- Wikipedia contributors. (2024). *Sieve of Eratosthenes*. Wikipedia. https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes  
-  Lab materials/prime_numbers.ipynb
# END TO TASK 4

## Task 5: Roots

### Explanation:
- This task focuses on isolating the fractional portion of each square root and expressing it as a 32-bit integer.
- First step is to take the square root of each prime.
- Then we only keep the decimal part.
- Then that is multiplied by 2^32 and converted to an interger to get a 32-bit value.

In [180]:
def get_root_bits(primes):
    results = []
    for p in primes:
        root = math.sqrt(p)          # Get square root of the prime
        frac = root % 1              # Isolate the decimal part
        bits = int (frac * (2**32))  # Convert fractional part to 32-bit integer
        results.append(bits)
    return results

- This part finds prime numbers, get the root bits, and prints them as hex values.

In [181]:
# Generate a list of prime numbers 
primes = primes_trial_division(100)

# Get the 32-bit root fraction for each prime number
root_bits = get_root_bits(primes)

# Lopp through the results and prints each prime with its 32-bit fractional part.
for i, bits in enumerate(root_bits):
     print(f"Prime: {primes[i]} → 32-bit frac: {bits:#010x}")

Prime: 5 → 32-bit frac: 0x3c6ef372
Prime: 7 → 32-bit frac: 0xa54ff53a
Prime: 9 → 32-bit frac: 0x00000000
Prime: 11 → 32-bit frac: 0x510e527f
Prime: 13 → 32-bit frac: 0x9b05688c
Prime: 15 → 32-bit frac: 0xdf7bd629
Prime: 17 → 32-bit frac: 0x1f83d9ab
Prime: 19 → 32-bit frac: 0x5be0cd19
Prime: 21 → 32-bit frac: 0x9523ae45
Prime: 23 → 32-bit frac: 0xcbbb9d5d
Prime: 25 → 32-bit frac: 0x00000000
Prime: 27 → 32-bit frac: 0x32370b90
Prime: 29 → 32-bit frac: 0x629a292a
Prime: 31 → 32-bit frac: 0x9159015a
Prime: 33 → 32-bit frac: 0xbe9ba858
Prime: 35 → 32-bit frac: 0xea843464
Prime: 37 → 32-bit frac: 0x152fecd8
Prime: 39 → 32-bit frac: 0x3eb83056
Prime: 41 → 32-bit frac: 0x67332667
Prime: 43 → 32-bit frac: 0x8eb44a87
Prime: 45 → 32-bit frac: 0xb54cda58
Prime: 47 → 32-bit frac: 0xdb0c2e0d
Prime: 49 → 32-bit frac: 0x00000000
Prime: 51 → 32-bit frac: 0x2434a74b
Prime: 53 → 32-bit frac: 0x47b5481d
Prime: 55 → 32-bit frac: 0x6a8bfbea
Prime: 57 → 32-bit frac: 0x8cc1f315
Prime: 59 → 32-bit frac: 0xae5f

### Research Perspective: 
- This technique is used when designing ryptographic algorithms like **SHA-256** 
- These values look random, but they always come out the same — which is perfect for something that needs to be both secure and reliable.
# END TO TASK 5

## Task 6: Proof of Work

## Setup: 
- Important word.txt from previous project.

## Explanation:
- This task finds the word in the English language that has the most `0` bits at the **start of its SHA-256 
hash**.
- SHA-256 is a secure hash function used in cryptography and blockchain.
- This idea is part of the “proof of work” system used in things like Bitcoin mining.


In [182]:
def leading_zeros(word):
    hash_hex = hashlib.sha256(word.encode()).hexdigest()
    hash_bin = bin(int(hash_hex, 16))[2:].zfill(256)
    return len(hash_bin) - len(hash_bin.lstrip('0'))  

In [183]:
with open("words.txt") as f:
    word_list = [line.strip() for line in f if line.strip().isalpha()]

# Track the best result
best_word = ""
max_zeros = 0

for word in word_list:
    zeros = leading_zeros(word)
    if zeros > max_zeros:
        best_word = word
        max_zeros = zeros

In [184]:
print(f"Best word: '{best_word}' with {max_zeros} leading zero bits in SHA-256 hash.")

Best word: 'APPLICANT' with 16 leading zero bits in SHA-256 hash.


# END TO TASK 6

## Task 7: Turing Machines

This Turing Machine adds '1' to a binary number.
It starts reading from the first digit on the left, and the last digit on the right is the smallest in value.

Example:
- Input: '100111'
- Output: '101000'

This table is what defines the rules of the Turing Machine.
- It will start in the start state and then moves right across the taper unitl it reaches the end (_).
- It then switches to the caryy state to handle the binary addition of +1. Flipping bits as needed.
- The machines comes to a halt when the correct results is written.

In [185]:
state_table = {
    # In the 'start' state: move right until we hit the end
    ('start', '0'): ('start', '0', 'R'), # just skip past 0
    ('start', '1'): ('start', '1', 'R'), # skip past 1 too
    ('start', '_'): ('carry', '_', 'L'),
    
    # In the 'carry' state: this is where we handle the actual adding part 
    ('carry', '0'): ('halt', '1', 'N'), 
    ('carry', '1'): ('carry', '0', 'L'),
    ('carry', '_'): ('halt', '1', 'N'),
}

This function prepares the Turing Machine's tape for processing:
- Takes the binary input and splits it into a list of chars.
- At the end a _ symbol is added so the machiune knows when to finish.
- The machine begins in the 'start' state.

In [186]:
# Set up the tape, head position, and starting state
def initialize_tape(input_str):
    # Convert the input string into a list of characters and add a blank symbol (_) at the end
    tape = list(input_str) + ['_']

     # Head starts at the beginning of the tape
    head = 0

    # Start the Turing machine in the 'start' state
    state = 'start'

    return tape, head, state

This function is what drives the main logic of the Turing Machine.
- Reads and wrties symbols on the tape and moves the head left or right according to the state_table.
- Machine stops when it reaches 'halt'.
- Show how real Turing Machines process inputs and evolve over a tape with set rules.

In [187]:
def run_turing_machine(state_table, tape, head, state):
    # Keeps going until it hits the 'halt' state
    while state != 'halt':
        symbol = tape[head]

        # No rules for current state = stop
        if (state, symbol) not in state_table:
            print("No transitioin defined for:", (state, symbol))
            break


        new_state, new_symbol, direction = state_table[(state, symbol)]
        
        # Write the new symbol on the tape
        tape[head] = new_symbol

        # Change to the new state
        state = new_state

        # Move head left or right
        if direction == 'R':
            head += 1
        elif direction == 'L':
            head -= 1
        
        # If head went off the left side, add a blank and reset head
        if head < 0:
            tape = ['_'] + tape
            head = 0

        # If head goes beyond the end, add a blank to the right
        elif head >= len(tape):
            tape.append('_')
            
    return tape

- This function acts as the main controlled, it pprepares the tape and state, runs the Turing machine, and prints the final result. 

In [188]:
def simulate(state_table, input_str):
    tape, head, state = initialize_tape(input_str)

    # Runs the Turing machine using the state table and input
    final_tape = run_turing_machine(state_table, tape, head, state)

    # Prints the final results, cleaned up
    print("Result:", ''.join(final_tape).strip('_'))

- Runs the machine on the binary input of '100111' to verify that the machine correctly adds 1, produces '101000'.

In [189]:
# Run the machine with the binary number '100111' for tests
simulate(state_table, '100111')

Result: 101000



References:
[Wikipedia definition of Turing machines](https://en.wikipedia.org/wiki/Turing_machine#Formal_definition)  


### Research Perspective: 


# END TO TASK 7

## Task 8: Computational Complexity
This tasks tests how many comparisions bubble sort makes when sorting different orders of the list '[1, 2, 3, 4, 5]'.

In [190]:
# List used to test
L = [1, 2, 3, 4, 5]

In [191]:
def bubble_sort(arr):
    count = 0
    a = arr.copy()
    n = len(a)
    for i in range(n):
       for j in range(0, n - i - 1):
            count += 1
            if a[j] > a[j + 1]:
                a[j], a[j + 1] = a[j + 1], a[j]
    return count

In [192]:
for p in permutations(L):
    comparisons = bubble_sort(list(p))
    print(f"{p} -> {comparisons} comparisons")


(1, 2, 3, 4, 5) -> 10 comparisons
(1, 2, 3, 5, 4) -> 10 comparisons
(1, 2, 4, 3, 5) -> 10 comparisons
(1, 2, 4, 5, 3) -> 10 comparisons
(1, 2, 5, 3, 4) -> 10 comparisons
(1, 2, 5, 4, 3) -> 10 comparisons
(1, 3, 2, 4, 5) -> 10 comparisons
(1, 3, 2, 5, 4) -> 10 comparisons
(1, 3, 4, 2, 5) -> 10 comparisons
(1, 3, 4, 5, 2) -> 10 comparisons
(1, 3, 5, 2, 4) -> 10 comparisons
(1, 3, 5, 4, 2) -> 10 comparisons
(1, 4, 2, 3, 5) -> 10 comparisons
(1, 4, 2, 5, 3) -> 10 comparisons
(1, 4, 3, 2, 5) -> 10 comparisons
(1, 4, 3, 5, 2) -> 10 comparisons
(1, 4, 5, 2, 3) -> 10 comparisons
(1, 4, 5, 3, 2) -> 10 comparisons
(1, 5, 2, 3, 4) -> 10 comparisons
(1, 5, 2, 4, 3) -> 10 comparisons
(1, 5, 3, 2, 4) -> 10 comparisons
(1, 5, 3, 4, 2) -> 10 comparisons
(1, 5, 4, 2, 3) -> 10 comparisons
(1, 5, 4, 3, 2) -> 10 comparisons
(2, 1, 3, 4, 5) -> 10 comparisons
(2, 1, 3, 5, 4) -> 10 comparisons
(2, 1, 4, 3, 5) -> 10 comparisons
(2, 1, 4, 5, 3) -> 10 comparisons
(2, 1, 5, 3, 4) -> 10 comparisons
(2, 1, 5, 4, 3

**References:**

- Wikipedia contributors. *Bubble sort*.[https://en.wikipedia.org/wiki/Bubble_sort](https://en.wikipedia.org/wiki/Bubble_sort).
- Lab at materials/computational_complexity.ipynb.
